code from Feb 28 
currently having issues with subprocess check_output returning non-zero exit status when creating the html


03.25.19 Check out https://github.com/hannal/PyPDF2/tree/master/PDF_Samples

03.25.19 http://easygui.sourceforge.net/tutorial.html#easygui-s-demonstration-routine


## Blog Post

### How to open a dialog box

### How to run a shell command from python
```python
# run a shell command from python and get the response
# as a string into a varaiable which you can print or discard
from subprocess import check_output
response = check_output("dir C:", shell=True)
print (response)
```


#### How to convert a Jupyter Notebook to HTML using nbconvert
##### verify that you have nbconvert installed as part of Anaconda
```cmd
jupyter nbconvert --help
```

The simplest way to use nbconvert is
    > jupyter nbconvert mynotebook.ipynb
which will convert mynotebook.ipynb to the default format (probably HTML).

You can specify the export format with `--to`.
Options include ['asciidoc', 'custom', 'html', 'latex', 'markdown', 'notebook', 'pdf', 'python', 'rst', 'script', 'slides']



In [6]:
import os

from subprocess import check_output
response = check_output(r'C:\Program Files\Git\git-bash.exe', shell=True)
#cmd = "C:\Program Files\Git\git-bash.exe --cd-to-home"
#response = os.system(cmd)
print (response)


b''


In [9]:
# https://www.sharats.me/posts/the-ever-useful-and-neat-subprocess-module/
import subprocess
subprocess.Popen(r'C:\Program Files\Git\git-bash.exe')

# TODO: opens in a background process - no GUI
# subprocess.Popen(r'"C:\Program Files\JetBrains\IntelliJ IDEA 2018.3.2\bin\idea64.exe"')

# TODO: use Se to launch canvas, penzu, github, etc..

### Random notes

must install python grip... add notes explaining this

https://nbconvert.readthedocs.io/en/latest/index.html

https://www.blog.pythonlibrary.org/2018/10/09/how-to-export-jupyter-notebooks-into-other-formats/

In [ ]:
'''
# Generating the TOC
# Use the directory structure to generate the TOC.... it's easier than going through and 
# looking for tags - especially in JSON. 
# THe code below is an example of an atttempt to search the files using regex.

import sys
import re
import getopt

def generateTOC(filename):
    pattern = "#"
    bufsize = 64 * 1024
    reflags = 0 | re.IGNORECASE

    try:
        prog = re.compile(pattern, reflags)
    except re.error as msg:
        usage("error in regular expression: %s" % msg)


    try:
        f = open(filename)
    except IOError as msg:
        usage("can't open %r: %s" % (filename, msg), 1)

    f.seek(0, 2)
    pos = f.tell()
    leftover = None
    while pos > 0:
        size = min(pos, bufsize)
        pos = pos - size
        f.seek(pos)
        buffer = f.read(size)
        lines = buffer.split("\n")
        del buffer
        if leftover is None:
            if not lines[-1]:
                del lines[-1]
        else:
            lines[-1] = lines[-1] + leftover
        if pos > 0:
            leftover = lines[0]
            del lines[0]
        else:
            leftover = None
        for line in reversed(lines):
            if prog.search(line):
                print(line)


                
fn = r'C:\Temp\aaa\My Book\content\Chapter 2\Section 1\Python+Fundamentals.ipynb'
generateTOC(fn)    
'''

In [ ]:
## Writing to one pdf
# https://stackoverflow.com/questions/3444645/merge-pdf-files

# Check out PDF Cat: https://pythonhosted.org/PyPDF2/Easy%20Concatenation%20Script.html 

import PyPDF2
import glob
from PyPDF2 import PdfFileMerger
base_dir = r'C:/Temp/aaa/My Book/content/*.pdf'
base_dir = r'C:/Temp/aaa/ExamplePDFs/*.pdf'

def make_one_pdf(base_dir):
    all_pdfs = glob.glob(base_dir)
    print(all_pdfs)
    if not all_pdfs:
        raise SystemExit('No pdf file found!')

    merger = PyPDF2.PdfFileMerger()
# https://github.com/mstamy2/PyPDF2/blob/master/PyPDF2/merger.py
# http://prodissues.com/2015/09/combine-multiple-pdf-files-into-one.html
    print("all_pdfs[0] is " + all_pdfs[0])
    with open(all_pdfs[0], 'rb') as first_obj:
        reader = PyPDF2.PdfFileReader(first_obj)
        print(reader.getNumPages())
        merger.append(all_pdfs[0],pages = (1,reader.getNumPages()))

    for pdf in all_pdfs[1:]:
        with open(pdf, 'rb') as obj:
            reader = PyPDF2.PdfFileReader(obj)
            print(reader.getNumPages())
            merger.append(fileobj=obj, pages=(1, reader.getNumPages()))

    with open(r'C:/Temp/aaa/output.pdf', 'wb') as f:
        merger.write(f) 
        
make_one_pdf(base_dir)        

In [ ]:
# loop through a collection of notebooks and md files
# Tkinter is the easiest way to open a file dialog box 
# if you don't want to have any other dependencies.
import os
import shutil
import glob
import PyPDF2
from PyPDF2 import PdfFileMerger
import pdfkit # requires https://wkhtmltopdf.org/downloads.html
import tkinter as tk
from tkinter import filedialog

# set properties for the directory open dialog box
root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.lift()
root.withdraw()

# for OSX
# self.root.lift()
# self.root.call('wm', 'attributes', '.', '-topmost', True)
# self.root.after_idle(self.root.call, 'wm', 'attributes', '.', '-topmost', False)

###################################################
# create TOC (use directory structure and filenames)
def createTOC(TOC):
    global fh 
    fh = open(TOC,"a")
    fh.write("<html><body>")
    fh.close()
def closeTOC(TOC):
    global fh 
    fh = open(TOC,"a")
    fh.write("</body></html>")
    fh.close()    
def appendTOC(TOC,level,text):
    global fh
    fh=open(TOC, "a")
    if level == 0:
        fh.write("{0}\n".format(text))
    if level == 1:
        fh.write("\t{0}\n".format(text))
    if level == 2:
        fh.write("\t\t{0}\n".format(text))
    fh.close()
###################################################
def make_one_pdf(base_dir):
    all_pdfs = glob.glob(base_dir + "/*.pdf")
    if not all_pdfs:
        raise SystemExit('No pdf file found!')

    merger = PyPDF2.PdfFileMerger()

    with open(all_pdfs[0], 'rb') as first_obj:
        merger.append(first_obj)

    for pdf in all_pdfs[1:]:
        with open(pdf, 'rb') as obj:
            reader = PyPDF2.PdfFileReader(obj)
            merger.append(fileobj=obj, pages=(1, reader.getNumPages()))

    with open('merge-pdfs.pdf', 'wb') as f:
        merger.write(f) 
                  
###################################################            
print("Select the Directory containing Jupyter Notebooks you wish to convert")
path = filedialog.askdirectory()
TOC = ''
for root, directories, filenames in os.walk(path):
    if len(TOC) == 0:
        #delete existing TOC so you can start new
        f = os.path.join(root,"TableOfContents.html")
        if os.path.exists(f):
            os.unlink(f)
        TOC = os.path.join(root,"TableOfContents.html")
        createTOC(TOC)
    else:
        appendTOC(TOC,0,root)
    
    #for directory in directories:
    for filename in filenames:
        name, ext = os.path.splitext(filename)
        infile = os.path.join(root,filename)
        if ext == ".html":
            os.unlink(infile)
        else:
            outfile = os.path.join(root,filename)
            outfile += ".html"
            # shutil.copy(file_path, destination_folder)
        
# TODO: Replace os.system with subprocess:
# https://stackoverflow.com/questions/4813238/difference-between-subprocess-popen-and-os-system
            if filename.endswith("ipynb"):    
                cmd = "jupyter nbconvert --output \"{0}\" --to html --template basic \"{1}\"".format(outfile,infile)
                os.system(cmd)                
                # add filename to TOC
                appendTOC(TOC,1,filename)
            
            if filename.endswith("md"):
                cmd = "grip \"{0}\" --export \"{1}\"".format(infile,outfile)
                os.system(cmd)
                # add filename to TOC
                appendTOC(TOC,1,filename)

closeTOC(TOC)
print ("done converting .ipynb and .md to .html")

print ("making PDFs")
# TODO: fix images - they don't show.
# TODO: Is wkhtmltopdf in the path?
path_wkthmltopdf = r'wkhtmltopdf/bin/wkhtmltopdf.exe'
config = pdfkit.configuration(wkhtmltopdf=path_wkthmltopdf)
for root, directories, filenames in os.walk(path):
    for filename in filenames:
        name, ext = os.path.splitext(filename)
        infile = os.path.join(root,filename)
        print(infile)
        if ext == ".html":
            outfile = os.path.join(path,filename + ".PDF")
            pdfkit.from_file(infile,outfile)
            pdfkit.from_file(infile,outfile,configuration=config)
print ("done making PDFs")
# push to github



In [5]:
 #!pip install easygui
import easygui
from easygui import *
ans = easygui.ynbox('Shall I continue?', 'Title', ('Yes', 'No'))
print(ans)
ans = easygui.msgbox('This is a basic message box.', 'Title Goes Here')
print(ans)
ans = easygui.buttonbox('Click on your favorite flavor.', 'Favorite Flavor', ('Chocolate', 'Vanilla', 'Strawberry'))
print(ans)

msgbox("Hello, world!")

True
OK
Chocolate


'OK'

In [ ]:
'''
djw 03.25.19 pdf's are not working..... 
#create individual PDFs for each topic
# TODO: fix images - they don't show.
# TODO: Is wkhtmltopdf in the path?
# TODO: prompt to find file
path_wkthmltopdf = r'C:\Users\GBTC440001ur\djw\jupyterbook\wkhtmltopdf\bin\wkhtmltopdf.exe'
config = pdfkit.configuration(wkhtmltopdf=path_wkthmltopdf)

for root, directories, filenames in os.walk(path):
    for filename in filenames:
        name, ext = os.path.splitext(filename)
        infile = os.path.join(root,filename)
        print(infile)
        if ext == ".html":  
            outfile = os.path.join(path,filename + ".pdf")
            print(outfile)
            pdfkit.from_file(infile,outfile,configuration=config)
make_one_pdf(r'C:\Temp\aaa\My Book\content')
print ("done making PDFs")
'''